In [7]:
import praw
import warnings
warnings.filterwarnings('ignore')

In [8]:
reddit = praw.Reddit(client_id='8kRfdrPdpbJe9Q',
                     client_secret='qeSfFi6kizAdurAFL790ZhQL_uM',
                     user_agent='Anukriti Jain')

In [9]:
def to_string(value):
    return "%s"%value

In [10]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


In [11]:
import pandas as pd
from datetime import datetime
posts = []
subreddit = reddit.subreddit('India')
for flair in flairs:
    for post in subreddit.search(flair, limit=300):
        count = 1
        post.comments.replace_more(limit=None)
        comment = ""
        for top_level_comment in post.comments.list():
            if(count<=5):
                comment = comment + " " + top_level_comment.body
            else: break
            count+=1
        posts.append([post.id, flair, post.title, post.author, post.url, post.num_comments, post.selftext, datetime.fromtimestamp(post.created), comment])


In [12]:
posts = pd.DataFrame(posts, columns=['id', 'flair','title', 'author ','url', 'num_comments', 'body', 'timestamp', 'comments'])
posts.drop_duplicates(subset='id', keep = False, inplace=True)
posts.to_csv('Final_reddit_India.csv', index = False)

In [13]:
df = pd.read_csv('Final_reddit_India.csv')
df.to_json('Final_reddit_India.json', orient = 'records')

In [14]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
#import enchant

In [15]:
tokenizer = RegexpTokenizer(r'\w+')
stopwords_eng = set(stopwords.words('english'))
stemmer = PorterStemmer()
#d = enchant.Dict("en_US")

def getStemmedText(text):
    
    text = text.lower()
    post = text
    #post = text.split()
    #post = ""
    #for word in new_text:
    #   if d.check(word)==True:
    #        post = post + " " + word
    
    tokens = tokenizer.tokenize(post)
    new_tokens = []
    stemmed_tokens = []
    
    for token in tokens:
        if token not in stopwords_eng:
            new_tokens.append(token)
    
    for token in new_tokens:
        stemmed_tokens.append(stemmer.stem(token))
        
    cleaned_text = ' '.join(stemmed_tokens)
    return cleaned_text
    

In [16]:
features = ['title','url','body','comments']

X = posts[features]
Y = posts.flair

In [17]:
X['title'] = X['title'].apply(getStemmedText).apply(to_string)
X['body'] = X['body'].apply(getStemmedText).apply(to_string)
X['comments'] = X['comments'].apply(getStemmedText).apply(to_string)
X['url'] = X['url'].apply(getStemmedText).apply(to_string)
#X['author '] = X['author '].apply(to_string)

combined = X["title"] + X["comments"] + X["url"]+X["body"]#+ X["author "]
X = X.assign(combined = combined)

In [18]:
x = X.combined

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.2, random_state=18)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [21]:
nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print("Accuracy of MultiNomial Naive Bayes: ",accuracy_score(y_test, y_pred)*100)
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy of MultiNomial Naive Bayes:  63.67432150313152
                    precision    recall  f1-score   support

          AskIndia       0.89      0.55      0.68        44
     Non-Political       0.55      0.43      0.48        49
     [R]eddiquette       0.42      0.71      0.52        38
         Scheduled       0.83      0.73      0.78        55
       Photography       1.00      0.31      0.47        36
Science/Technology       0.48      0.75      0.58        28
          Politics       0.75      0.60      0.67        50
  Business/Finance       0.62      0.67      0.64        51
    Policy/Economy       0.64      0.70      0.67        43
            Sports       0.59      0.80      0.68        46
              Food       0.73      0.88      0.80        34
               AMA       0.00      0.00      0.00         5

          accuracy                           0.64       479
         macro avg       0.62      0.59      0.58       479
      weighted avg       0.68      0.64   

In [22]:
from sklearn.linear_model import LogisticRegression

logr = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=-1, C=1e5, random_state=18)),
                 ])
logr.fit(X_train, y_train)

y_pred = logr.predict(X_test)

print("Accuracy of Logistic Regression: ", accuracy_score(y_pred, y_test)*100)
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy of Logistic Regression:  74.73903966597078
                    precision    recall  f1-score   support

          AskIndia       0.89      0.70      0.78        44
     Non-Political       0.68      0.69      0.69        49
     [R]eddiquette       0.58      0.76      0.66        38
         Scheduled       0.85      0.80      0.82        55
       Photography       0.84      0.72      0.78        36
Science/Technology       0.63      0.68      0.66        28
          Politics       0.79      0.68      0.73        50
  Business/Finance       0.69      0.75      0.72        51
    Policy/Economy       0.75      0.70      0.72        43
            Sports       0.71      0.89      0.79        46
              Food       0.94      0.88      0.91        34
               AMA       0.67      0.40      0.50         5

          accuracy                           0.75       479
         macro avg       0.75      0.72      0.73       479
      weighted avg       0.76      0.75      0

In [35]:
from sklearn.ensemble import RandomForestClassifier
  
randfr = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 500, random_state = 18)),
                 ])
randfr.fit(X_train, y_train)

y_pred = randfr.predict(X_test)

print("Accuracy of Random Forest Classifier: ", accuracy_score(y_pred, y_test)*100)
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy of Random Forest Classifier:  81.00208768267223
                    precision    recall  f1-score   support

          AskIndia       0.87      0.77      0.82        44
     Non-Political       0.77      0.88      0.82        49
     [R]eddiquette       0.77      0.79      0.78        38
         Scheduled       0.89      0.76      0.82        55
       Photography       0.83      0.83      0.83        36
Science/Technology       0.67      0.79      0.72        28
          Politics       0.84      0.74      0.79        50
  Business/Finance       0.70      0.82      0.76        51
    Policy/Economy       0.89      0.74      0.81        43
            Sports       0.82      0.91      0.87        46
              Food       0.89      0.94      0.91        34
               AMA       1.00      0.40      0.57         5

          accuracy                           0.81       479
         macro avg       0.83      0.78      0.79       479
      weighted avg       0.82      0.81  

In [40]:
from sklearn.neural_network import MLPClassifier
  
mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,20,20), random_state=18)),
                 ])
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print("Accuracy of MLP Classifier: ", accuracy_score(y_pred, y_test)*100)
print(classification_report(y_test, y_pred,target_names=flairs))

Accuracy of MLP Classifier:  51.98329853862212
                    precision    recall  f1-score   support

          AskIndia       0.86      0.41      0.55        44
     Non-Political       0.30      0.57      0.39        49
     [R]eddiquette       0.43      0.61      0.51        38
         Scheduled       0.84      0.65      0.73        55
       Photography       0.41      0.25      0.31        36
Science/Technology       0.50      0.43      0.46        28
          Politics       0.74      0.58      0.65        50
  Business/Finance       0.43      0.39      0.41        51
    Policy/Economy       0.54      0.65      0.59        43
            Sports       0.66      0.41      0.51        46
              Food       0.46      0.76      0.58        34
               AMA       1.00      0.20      0.33         5

          accuracy                           0.52       479
         macro avg       0.60      0.49      0.50       479
      weighted avg       0.58      0.52      0.52  

In [44]:
nb = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                 ('tfidf', TfidfTransformer()),
                 ('clf', SVC(kernel ='linear', random_state=18)),
                ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print("Accuracy of Support Vector Classifier: ", accuracy_score(y_test, y_pred)*100)
print(classification_report(y_test, y_pred, target_names=flairs))

Accuracy of Support Vector Classifier:  72.44258872651356
                    precision    recall  f1-score   support

          AskIndia       0.88      0.64      0.74        44
     Non-Political       0.49      0.76      0.59        49
     [R]eddiquette       0.54      0.74      0.62        38
         Scheduled       0.89      0.73      0.80        55
       Photography       0.85      0.61      0.71        36
Science/Technology       0.76      0.68      0.72        28
          Politics       0.78      0.70      0.74        50
  Business/Finance       0.71      0.80      0.75        51
    Policy/Economy       0.85      0.67      0.75        43
            Sports       0.71      0.87      0.78        46
              Food       0.93      0.76      0.84        34
               AMA       1.00      0.40      0.57         5

          accuracy                           0.72       479
         macro avg       0.78      0.70      0.72       479
      weighted avg       0.76      0.72 

In [45]:
import pickle
filename = 'model.sav'
pickle.dump(randfr, open(filename, 'wb'))